<a href="https://colab.research.google.com/github/preetamjumech/Bayesian-Search-Optimization/blob/main/Preetam_Saha__Bayesian_Search__31_05_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00551/pp_gas_emission.zip

--2022-05-31 11:25:52--  https://archive.ics.uci.edu/ml/machine-learning-databases/00551/pp_gas_emission.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1090294 (1.0M) [application/x-httpd-php]
Saving to: ‘pp_gas_emission.zip.1’

pp_gas_emission.zip 100%[===================>]   1.04M  6.08MB/s    in 0.2s    

2022-05-31 11:25:53 (6.08 MB/s) - ‘pp_gas_emission.zip.1’ saved [1090294/1090294]



In [ ]:
from hyperopt import fmin,hp,STATUS_OK,tpe,Trials
from hyperopt.pyll.base import scope
import hyperopt

In [ ]:
!unzip pp_gas_emission.zip

Archive:  pp_gas_emission.zip
replace gt_2011.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace gt_2012.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace gt_2013.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace gt_2014.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace gt_2015.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
gt=pd.read_csv("gt_2015.csv")
gt.head()

,AT,AP,AH,AFDP,GTEP,TIT,TAT,TEY,CDP,CO,NOX
0,1.95320,1020.1,84.985,2.5304,20.116,1048.7,544.92,116.27,10.799,7.4491,113.250
1,1.21910,1020.1,87.523,2.3937,18.584,1045.5,548.50,109.18,10.347,6.4684,112.020
2,0.94915,1022.2,78.335,2.7789,22.264,1068.8,549.95,125.88,11.256,3.6335,88.147
3,1.00750,1021.7,76.942,2.8170,23.358,1075.2,549.63,132.21,11.702,3.1972,87.078
4,1.28580,1021.6,76.732,2.8377,23.483,1076.2,549.68,133.58,11.737,2.3833,82.515


In [ ]:
gt_new=gt.loc[:,"AT":"CO"]
gt_new.shape

(7384, 10)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor,RandomForestRegressor,GradientBoostingRegressor
from xgboost import XGBRegressor

In [ ]:
lin_reg= LinearRegression()
dt_reg=DecisionTreeRegressor()
#ada_reg=AdaBoostRegressor()
gbm_reg=GradientBoostingRegressor()
rf_reg=RandomForestRegressor()
xgb_reg = XGBRegressor()

In [ ]:
param_dt = {
    'max_depth':scope.int(hp.quniform('max_depth',2,20,1)),
    'ccp_alpha':hp.uniform('ccp_alpha',0.001,0.1)
}
param_rf = {
    'n_estimators':scope.int(hp.quniform('n_estimators',50,500,1)),
    'max_features':hp.choice('max_features',list(range(2,7)))
}
param_gbm = {
    'max_depth':scope.int(hp.quniform('max_depth',1,6,1)),
    'n_estimators':scope.int(hp.quniform('n_estimators',50,500,1)),
    'learning_rate':hp.uniform('learning_rate',0.001,0.1)
}
param_xgb={
    'max_depth':scope.int(hp.quniform('max_depth',1,6,1)),
    'n_estimators':scope.int(hp.quniform('n_estimators',50,500,1)),
    'learning_rate':hp.uniform('learning_rate',0.001,0.1),
    'colsample_bytree':hp.uniform('colsample_bytree',0.2,0.8)
}

In [ ]:
def rmse(y_act,y_pred):
  return (np.sqrt(np.mean((y_act-y_pred)**2)))

In [ ]:
from sklearn.metrics import make_scorer,mean_squared_error
rmse=make_scorer(rmse,greater_is_better=False)

In [ ]:
from sklearn.model_selection import train_test_split,cross_val_score

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(gt_new.loc[:,"AT":"CDP"],gt_new.loc[:,"CO"],test_size=0.3,random_state=123)

In [ ]:
def hp_obj_func_dt(param):
  dt= DecisionTreeRegressor(**param)
  out=cross_val_score(dt,X=X_train,y=y_train,scoring=rmse,cv=5).mean()
  return out

In [ ]:
def surr_func_dt(params):
  perf=hp_obj_func_dt(params)
  return{'loss':-perf,'status':STATUS_OK}

In [ ]:
trials=Trials()
best_dt = fmin(surr_func_dt, space=param_dt,algo=tpe.suggest,max_evals=10,trials=trials)
print("best_dt:")
print(best_dt)

100%|██████████| 10/10 [00:02<00:00,  3.37it/s, best loss: 1.4434945179780703]
best_dt:
{'ccp_alpha': 0.00407092224976983, 'max_depth': 2.0}


In [ ]:
dt_reg=DecisionTreeRegressor(**best_dt)
dt_reg.fit(X_train,y_train)
pred_dt=dt_reg.predict(X_test)
print(mean_squared_error(pred_dt,y_test))

1.5422191901119466


In [ ]:
from sklearn.linear_model import LinearRegression
lin_reg=LinearRegression()
lin_reg.fit(X_train,y_train)
pred_lin_reg=lin_reg.predict(X_test)
print(mean_squared_error(pred_lin_reg,y_test))

1.1916144779735578
